<a href="https://colab.research.google.com/github/vinaysoma100/Coursera_Capstone/blob/master/Capstone_Project_The_Battle_of_Neighborhoods(week%201).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Introduction: Theatre opening in new delhi

In such a large and rich city ,new delhi,india, it will be competitive to start up business. in this project  we will just discuss about predicting hot spot

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np
!pip install folium
import folium 

2.First of all we need to collect Data


3.We import all the tools we need.

Apply your credential ID on [FourSquare]

In [36]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


Get requests near new delhi

In [16]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "theater",
    "near": "new delhi",
    "radius": 1000,
    "limit": 100}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

Transform data into json then request geocode.

In [37]:
d = data.json()["response"]
d.keys()


dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

In [18]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'New Delhi', 'New Delhi')

In [19]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 28.64350044264776, 'lng': 77.23157336581968},
  'sw': {'lat': 28.62745985200316, 'lng': 77.21531534538968}},
 75)

In [20]:
d["geocode"]

{'cc': 'IN',
 'center': {'lat': 28.63576, 'lng': 77.22445},
 'displayString': 'New Delhi, National Capital Territory of Delhi, India',
 'geometry': {'bounds': {'ne': {'lat': 28.883565, 'lng': 77.346493},
   'sw': {'lat': 28.404122, 'lng': 76.838878}}},
 'longId': '72057594039189417',
 'slug': 'new-delhi-india',
 'what': '',
 'where': 'new delhi'}

We start creating group including information which is recommended.

In [22]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [23]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

Creating items of objects  shop and their attributes - id, address, name, etc

In [25]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 75


{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-54783eab498e910f8bd1781d-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1c9941735',
    'name': 'Ice Cream Shop',
    'pluralName': 'Ice Cream Shops',
    'primary': True,
    'shortName': 'Ice Cream'}],
  'id': '54783eab498e910f8bd1781d',
  'location': {'address': 'L Block Outer Circle',
   'cc': 'IN',
   'city': 'New Delhi',
   'country': 'India',
   'crossStreet': 'Connaught Place',
   'formattedAddress': ['L Block Outer Circle (Connaught Place)',
    'New Delhi',
    'Delhi',
    'India'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 28.634455305513185,
     'lng': 77.22213885818994}],
   'lat': 28.634455305513185,
   'lng': 77.22213885818994,
   'state': 'Delhi'},
  'name': 'Naturals Ice Cream',
  'pho

In [26]:
items[1]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4cb876d7f50e224bd2d6e6fb-1',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d10f941735',
    'name': 'Indian Restaurant',
    'pluralName': 'Indian Restaurants',
    'primary': True,
    'shortName': 'Indian'}],
  'id': '4cb876d7f50e224bd2d6e6fb',
  'location': {'address': 'K Block',
   'cc': 'IN',
   'city': 'New Delhi',
   'country': 'India',
   'crossStreet': 'Connaught Place',
   'formattedAddress': ['K Block (Connaught Place)',
    'New Delhi',
    'Delhi',
    'India'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 28.635487253836967,
     'lng': 77.22064969494076}],
   'lat': 28.635487253836967,
   'lng': 77.22064969494076,
   'state': 'Delhi'},
  'name': 'Sagar Ratna',
  'photos': {'count': 0, 'groups': []}

From the output we can identify necessary factors of what we will use later to consider the probability of launching our up-to-coming location.

Based on that we start to organize what we have got.

In [27]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i theaters" % len(df))
df.head()

{'address': 'L Block Outer Circle', 'crossStreet': 'Connaught Place', 'lat': 28.634455305513185, 'lng': 77.22213885818994, 'labeledLatLngs': [{'label': 'display', 'lat': 28.634455305513185, 'lng': 77.22213885818994}], 'cc': 'IN', 'city': 'New Delhi', 'state': 'Delhi', 'country': 'India', 'formattedAddress': ['L Block Outer Circle (Connaught Place)', 'New Delhi', 'Delhi', 'India']}
{'address': 'K Block', 'crossStreet': 'Connaught Place', 'lat': 28.635487253836967, 'lng': 77.22064969494076, 'labeledLatLngs': [{'label': 'display', 'lat': 28.635487253836967, 'lng': 77.22064969494076}], 'cc': 'IN', 'city': 'New Delhi', 'state': 'Delhi', 'country': 'India', 'formattedAddress': ['K Block (Connaught Place)', 'New Delhi', 'Delhi', 'India']}
{'address': 'D-3', 'crossStreet': 'Near BIG Odeon Cinema', 'lat': 28.634202294199614, 'lng': 77.22077980678422, 'labeledLatLngs': [{'label': 'display', 'lat': 28.634202294199614, 'lng': 77.22077980678422}], 'postalCode': '110001', 'cc': 'IN', 'city': 'New De

,uid,name,shortname,address,postalcode,lat,lng
0,4f0f00a7e4b07eb2379b1e1d,Khan Chacha | खान चाचा | خان چاچا,Indian,,110001,28.634202,77.220780
1,4b489b54f964a520595026e3,Connaught Place | कनॉट प्लेस (Connaught Place),Plaza,,110001,28.632731,77.220018
2,4e1d611e45ddc696bb391c3b,Route 04,Bar,,110001,28.634890,77.220225
3,4bf107ab24f020a1e0a8684f,Nizam's Kathi Kabab | निजा़म काठी कबाब,Indian,,110001,28.634858,77.219462
4,4e4e44902271a1bdc3ccee8d,Wenger's,Bakery,,110001,28.633412,77.218292


As we can see that there are many shops without address we need to execute hasattr() to determine if each object (coffeeshop) has a attribute (address).

Next step we will execute a very important part - get coordinates of Newdelhi and create folium map which will help visualize what we have got from data.

In [28]:
newdelhi_center = d["geocode"]["center"]
newdelhi_center

{'lat': 28.63576, 'lng': 77.22445}

In [32]:
df.head(20)

,uid,name,shortname,address,postalcode,lat,lng
0,4f0f00a7e4b07eb2379b1e1d,Khan Chacha | खान चाचा | خان چاچا,Indian,,110001,28.634202,77.220780
1,4b489b54f964a520595026e3,Connaught Place | कनॉट प्लेस (Connaught Place),Plaza,,110001,28.632731,77.220018
2,4e1d611e45ddc696bb391c3b,Route 04,Bar,,110001,28.634890,77.220225
3,4bf107ab24f020a1e0a8684f,Nizam's Kathi Kabab | निजा़म काठी कबाब,Indian,,110001,28.634858,77.219462
4,4e4e44902271a1bdc3ccee8d,Wenger's,Bakery,,110001,28.633412,77.218292
5,4b59921bf964a520fe8c28e3,United Coffee House,Indian,,110001,28.632477,77.221120
6,519ba450498eb0c559152d94,HOTEL SARAVANA BHAVAN,South Indian,,110001,28.632319,77.216445
7,4cbb2592dd41a35df95eeca0,Rajdhani Thali,Indian,,110001,28.629999,77.220401
8,54440189498e4795b1111834,Immigrants Project - A Cafe in History,Café,,110001,28.634055,77.218867
9,4e2ab5758877b69d49d7c525,Fabindia,Apparel,,110001,28.632012,77.217729


In [29]:
from folium import plugins


map_newdelhi = folium.Map(location=[28.63576, 77.22445], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_newdelhi)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_newdelhi.add_child(plugins.HeatMap(hm_data))

map_newdelhi

By spotting the clusters of items we can see which neighborhood has density of  business.

3. Conclusion

We will need a location where we can catch out customers from "hot" location we have picked up from the map and stay in a certain distance so as to lessen the competivity of business.

In [30]:
lat = 28.63576
lng = 77.22445
map_newdelhi = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="O Day Roi!",
    color='green',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_newdelhi)
map_newdelhi

Look at the blue buble, here we find out that it will locate about the Deen dayal upadyay road. 